## Lab 9

### Connecting Python with MySQL

In [1]:
from sqlalchemy import create_engine
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

### Cheking number of rentals for each customer

In [9]:
query_may = '''
select customer_id, count(*) as rentals_may
from sakila.rentals_may
group by customer_id;
'''

data_may = pd.read_sql_query(query_may, engine)
data_may

,customer_id,rentals_may
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [10]:
query_june = '''
select customer_id, count(*) rentals_june
from sakila.rentals_june
group by customer_id;
'''

data_june = pd.read_sql_query(query_june, engine)
data_june

,customer_id,rentals_june
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


### Comparing june and may

In [22]:
cust_id_rental_may = data_may[data_may['customer_id'] == 600]
cust_id_rental_may

,customer_id,rentals_may


In [17]:
cust_id_rental_may['rentals_may'].values[0]

3

In [67]:
def comparing_rentals(df1, df2, input_customer):
    cust_rental_may = df1[df1['customer_id'] == input_customer]
    cust_rental_june = df2[df2['customer_id'] == input_customer]
    
    if cust_rental_june.shape[0] == 0 and cust_rental_may.shape[0] == 0:
        result = 'Invalid Customer'
        
    elif cust_rental_june.shape[0] == 0 and cust_rental_may.shape[0] != 0:
        value_may = cust_rental_may['rentals_may'].values[0]
        result = 'Customer only rented in may', value_may
        
    elif cust_rental_june.shape[0] != 0 and cust_rental_may.shape[0] == 0:
        value_june = cust_rental_june['rentals_june'].values[0]
        result = 'Customer only rented in june', value_june
        
    else:
        value_may = cust_rental_may['rentals_may'].values[0]
        value_june = cust_rental_june['rentals_june'].values[0]
        if value_may > value_june:
            result = 'More rentals in may', value_may
        elif value_june > value_may:
            result = 'More rentals in june', value_june
        else:
            result = 'Count of rentals is the same for both months', value_may
    return result

In [77]:
comparing_rentals(data_may, data_june, 20)

('More rentals in june', 4)

In [79]:
for i in range(30):
    print(comparing_rentals(data_may, data_june, i))
    i += i

Invalid Customer
('More rentals in june', 7)
('Count of rentals is the same for both months', 1)
('More rentals in june', 4)
('Customer only rented in june', 6)
('More rentals in june', 5)
('More rentals in june', 4)
('Count of rentals is the same for both months', 5)
('More rentals in june', 3)
('More rentals in may', 3)
('More rentals in june', 5)
('More rentals in june', 3)
('More rentals in june', 6)
('Customer only rented in june', 4)
('More rentals in may', 5)
('Customer only rented in june', 3)
('Count of rentals is the same for both months', 4)
('Count of rentals is the same for both months', 3)
('More rentals in june', 4)
('More rentals in may', 6)
('More rentals in june', 4)
('More rentals in june', 5)
('More rentals in may', 3)
('More rentals in june', 5)
('More rentals in june', 5)
('More rentals in june', 6)
('More rentals in june', 7)
('More rentals in june', 8)
('More rentals in june', 5)
('More rentals in june', 7)


### Other way to compare

In [82]:
data_comparison = data_may.merge(right=data_june, how='inner', on='customer_id')
data_comparison

,customer_id,rentals_may,rentals_june
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4
...,...,...,...
507,594,4,6
508,595,1,2
509,596,6,2
510,597,2,3


In [86]:
data_comparison.loc[0]

customer_id     1
rentals_may     2
rentals_june    7
Name: 0, dtype: int64

In [87]:
data_comparison.loc[0][1]

2

In [ ]:
# since it was made an inner join, there are no cases for a customer that only have rented movies in a single month.

In [90]:
def comparing_rentals_2(df, i):
    if df.loc[i][1] > df.loc[i][2]:
        result = 'Customer rented more movies in may', df.loc[i][1]
    elif df.loc[i][1] < df.loc[i][2]:
        result = 'Customer rented more movies in june', df.loc[i][2]
    else:
        result = 'Customer rented the same amount of movies in may and june', df.loc[i][1]
    print(result)

In [95]:
comparing_rentals_2(data_comparison, 7)

('Customer rented more movies in may', 3)
